<a href="https://colab.research.google.com/github/SwayamPal28/git_test/blob/main/prototype.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install pandas sentence-transformers
!pip install llama-cpp-python

In [ ]:
!pip install pyngrok
!pip install websockets
!pip install asyncio

In [ ]:
import websockets
import asyncio
import json

In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util
from llama_cpp import Llama
import torch

ModuleNotFoundError: No module named 'llama_cpp'

In [ ]:
import os
import socket
from pyngrok import ngrok, conf
from google.colab import userdata

In [ ]:
print("configuring ngrok ....")
try:
  authtoken = userdata.get('NGROK_AUTHTOKEN')
  conf.get_default().auth_token = authtoken
  print("Ngrok authtoken configured successfully!")
except Exception as e:
    print(f"Error configuring ngrok: {e}")
    print("Please ensure you've saved your authtoken as a Colab Secret named 'NGROK_AUTHTOKEN'.")

In [ ]:
!wget "https://huggingface.co/microsoft/Phi-3-mini-4k-instruct-gguf/resolve/main/Phi-3-mini-4k-instruct-q4.gguf"

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/acupuncture/acupressure(3).csv')
print(df.head())

In [ ]:
# df.dropna(axis=1, how='all', inplace=True)

In [ ]:
retriever_model = SentenceTransformer('all-MiniLM-L6-v2')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
keyword_list = df['Keywords'].tolist()
corpus_embedding = retriever_model.encode(keyword_list, convert_to_tensor=True)
# print(corpus_embedding)

In [ ]:
llm = Llama(
    model_path = '/content/Phi-3-mini-4k-instruct-q4.gguf',
    n_gpu_layers=-1,
    n_ctx=4096,
)

llama_model_loader: loaded meta data with 24 key-value pairs and 195 tensors from /content/Phi-3-mini-4k-instruct-q4.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = phi3
llama_model_loader: - kv   1:                               general.name str              = Phi3
llama_model_loader: - kv   2:                        phi3.context_length u32              = 4096
llama_model_loader: - kv   3:                      phi3.embedding_length u32              = 3072
llama_model_loader: - kv   4:                   phi3.feed_forward_length u32              = 8192
llama_model_loader: - kv   5:                           phi3.block_count u32              = 32
llama_model_loader: - kv   6:                  phi3.attention.head_count u32              = 32
llama_model_loader: - kv   7:               phi3.attention.head_count_kv u32  

In [1]:
# def generate_response(user_query):
#   confidence_threshold = 0.4
#   query_embedding = retriever_model.encode(user_query, convert_to_tensor=True)
#   cos_scores = util.cos_sim(query_embedding, corpus_embedding)[0]

#   highest_scores, best_match_index = torch.max(cos_scores, dim=0)
#   highest_scores = highest_scores.item()
#   best_match_index = best_match_index.item()

#   if(highest_scores > confidence_threshold):
#       match_data = df.iloc[best_match_index].to_dict()

#       prompt_content=f'''
#       Task: Create a concise and helpful response for a user based on the provided context. Follow these rules:
#       1. Start by identifying the user's likely symptom based on the context.
#       2. Recommend the specific reflex point and foot from the context and make sure to mention the number of the reflex point.
#       3. Provide 2-3 brief, general wellness tips.
#       4. End with a clear disclaimer to consult a professional.
#       5. Keep the entire response under 80 words.

#       Context:
#         -Symptom : {match_data['UseCaseTag']}
#         -Feet : {match_data['Feet']}
#         -Reflex Point : {match_data['ReflexPoints']}
#         -Point Index : {match_data['PointNumber']}

#       User query: {user_query}

#       Create a helpful response based on the context.
#       '''

#       completion = llm.create_chat_completion(
#           messages = [
#               {"role": "system", "content": "You are a helpful reflexology assistant who provides concise, structured answers."},
#               {"role": "user", "content": prompt_content}
#           ],
#           max_tokens = 150
#       )

#       return completion['choices'][0]['message']['content']

#   else:
#       return "I'm sorry, I don't have a specific recommendation for that. Could you please try rephrasing your symptom?"


import torch
from sentence_transformers import util

def generate_response(user_query, df, retriever_model, corpus_embedding, llm):
    """
    Finds all relevant contexts above a confidence threshold and generates a response.
    """
    confidence_threshold = 0.4
    query_embedding = retriever_model.encode(user_query, convert_to_tensor=True)
    cos_scores = util.cos_sim(query_embedding, corpus_embedding)[0]

    passing_indices = torch.where(cos_scores > confidence_threshold)[0]

    if len(passing_indices) > 0:
        all_contexts = []
        for idx in passing_indices:
            match_data = df.iloc[idx.item()].to_dict()
            context_entry = (
                f"  - Symptom: {match_data['UseCaseTag']}\n"
                f"  - Feet: {match_data['Feet']}\n"
                f"  - Reflex Point: {match_data['ReflexPoints']}\n"
                f"  - Point Index: {match_data['PointNumber']}"
            )
            all_contexts.append(context_entry)

        combined_context = "\n\n".join(all_contexts)

        prompt_content = f'''
        Task: Create a concise and helpful response for a user based on all the provided context entries. Follow these rules:
        1. Synthesize the information from all context entries.
        2. Identify the user's likely symptom(s) based on the query and context.
        3. Recommend all relevant reflex points, specifying the foot and point number for each.
        4. Provide 2-3 brief, general wellness tips.
        5. End with a clear disclaimer to consult a professional.
        6. Keep the entire response under 100 words.

        Contexts:
        {combined_context}

        User query: {user_query}

        Create a helpful, synthesized response based on the contexts.
        '''

        completion = llm.create_chat_completion(
            messages=[
                {"role": "system", "content": "You are a helpful reflexology assistant who provides concise, synthesized answers from multiple sources."},
                {"role": "user", "content": prompt_content}
            ],
            max_tokens=150
        )

        return completion['choices'][0]['message']['content']

    else:
        return "I'm sorry, I don't have a specific recommendation for that. Could you please try rephrasing your symptom?"

In [2]:
def find_best_match(user_query):
  confidence_threshold = 0.4
  query_embedding = retriever_model.encode(user_query, convert_to_tensor=True)
  cos_scores = util.cos_sim(query_embedding, corpus_embedding)[0]

  passing_indices = torch.where(cos_scores > confidence_threshold)[0]

  if len(passing_indices) == 0:
    return []

  matching_points = df.iloc[passing_indices.cpu().numpy()]["PointNumber"].tolist()

  return list(set(matching_points))

In [ ]:
# CONNECTED_CLIENTS = set()

# async def connection_handler(websocket):
#     CONNECTED_CLIENTS.add(websocket)
#     print(f"A client connected. Total clients: {len(CONNECTED_CLIENTS)}")
#     try:
#         await websocket.wait_closed()
#     finally:
#         CONNECTED_CLIENTS.remove(websocket)
#         print(f"A client disconnected. Total clients: {len(CONNECTED_CLIENTS)}")

# async def broadcast_result(point_number):
#     if CONNECTED_CLIENTS:
#         json_message = json.dumps([point_number])
#         print(f"Broadcasting: {json_message} to {len(CONNECTED_CLIENTS)} client(s)")
#         tasks = [client.send(json_message) for client in CONNECTED_CLIENTS]
#         await asyncio.gather(*tasks)

# async def query_loop():
#     """Keep asking for queries but only if clients are connected."""
#     while True:
#         if CONNECTED_CLIENTS:
#             user_query = input("Enter your query here: ")
#             point_number = find_best_match(user_query)

#             if point_number is not None:
#                 point_number = int(point_number)
#                 print(f"Match found. PointNumber = {point_number}")
#                 await broadcast_result(point_number)
#             else:
#                 print("No match found")
#         else:
#             await asyncio.sleep(1)

# async def main():
#     port = 8002

#     server = None
#     public_url = None

#     try:
#         server = await websockets.serve(connection_handler, "0.0.0.0", port)
#         # await asyncio.sleep(1)
#         public_url = ngrok.connect(port, "http")

#         print("="*50)
#         print("SERVER IS LIVE AND WAITING FOR CLIENTS")
#         print(f"Share this public address: {public_url.public_url}")
#         print("Press the STOP button in the cell to shut down.")
#         print("="*50)

#         # run query loop forever
#         await query_loop()

#     except KeyboardInterrupt:
#         print("\nShutdown signal received...")
#     finally:
#         print("Starting graceful shutdown.")
#         if public_url:
#             ngrok.disconnect(public_url.public_url)
#         if server:
#             server.close()
#             await server.wait_closed()
#         print("Server has been shut down gracefully.")

# await main()




CONNECTED_CLIENTS = set()

async def interactive_handler(websocket):
    print(f"✅ Client connected. Total clients: {len(CONNECTED_CLIENTS) + 1}")
    CONNECTED_CLIENTS.add(websocket)
    try:
        async for query in websocket:
            print(f"\n[Server] Received query: '{query}'")

            point_number_list = find_best_match(query)
            # point_number = int(point_number)

            if point_number_list is not None:
                json_response = json.dumps(point_number_list)

                await websocket.send(json_response)
                print(f"[Server] Sent response: {json_response}")
            else:
                error_response = json.dumps({"error": "No match found for that query."})
                await websocket.send(error_response)
                print(f"[Server] Sent response: {error_response}")

    except websockets.exceptions.ConnectionClosed:
        print("A client disconnected.")
    finally:
        CONNECTED_CLIENTS.remove(websocket)
        print(f"Client disconnected. Total clients: {len(CONNECTED_CLIENTS)}")

async def main():
    port = 8002
    server = None
    public_url = None

    try:
        server = await websockets.serve(interactive_handler, "0.0.0.0", port)
        public_url = ngrok.connect(port, "http")

        print("="*50)
        print("SERVER IS LIVE AND WAITING FOR CLIENTS")
        print(f"Share this public address: {public_url.public_url}")
        print("Press the STOP button in the cell to shut down.")
        print("="*50)

        await asyncio.Future()

    except KeyboardInterrupt:
        print("\nShutdown signal received...")
    finally:
        print("Starting graceful shutdown.")
        if public_url:
            ngrok.disconnect(public_url.public_url)
        if server:
            server.close()
            await server.wait_closed()
        print("Server has been shut down gracefully.")

await main()

SERVER IS LIVE AND WAITING FOR CLIENTS
Share this public address: https://28769fe2ea63.ngrok-free.app
Press the STOP button in the cell to shut down.
✅ Client connected. Total clients: 1

[Server] Received query: 'ahd trouble sleeping at night, frequent urination'
[Server] Sent response: [2, 7, 9, 10, 11, 14]

[Server] Received query: 'i had back pain and stinging sensation in arms'
[Server] Sent response: [2, 3, 4, 9, 11, 12, 13, 14]
Client disconnected. Total clients: 0
✅ Client connected. Total clients: 1

[Server] Received query: 'i had stinging sensation in my arms'
[Server] Sent response: [11, 3, 4, 12]
A client disconnected.
Client disconnected. Total clients: 0
✅ Client connected. Total clients: 1

[Server] Received query: 'i have posture issues'
[Server] Sent response: [2, 3, 4, 6, 13, 14]
Client disconnected. Total clients: 0


In [ ]:
user_query = "Didnt sleep right, feeling sore mostly in the back"
# response = generate_response(user_query)
print(find_best_match(user_query))

[14]
